In [50]:
import random
import json 
import pickle
import numpy as np
import json

import nltk
nltk.download('popular')
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import tensorflow as tf

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/test/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/test/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/test/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/test/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/test/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/test/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!

In [33]:
lemmatizer = WordNetLemmatizer()
intents =json.loads(open("intents.json").read())

words = []
classes = []
documents = []
ignore_letters = ["?", "!", ".",","]

for intent in intents["intents"]:
  for pattern in intent["patterns"]:
    word_list = nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list,intent["tag"]))
    if intent["tag"] not in classes:
      classes.append(intent["tag"])
  

In [34]:
words = sorted(set([lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]))

In [35]:
classes = sorted(set(classes))
pickle.dump(words, open("words.pkl","wb"))
pickle.dump(words, open("classes.pkl","wb"))

In [56]:
training = []
output_empty = [0] * len(classes)

for document in documents:
  bag = []
  word_patterns = document[0]
  words_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  
  [bag.append(1) if word in words_patterns else bag.append(0) for word in words]
  
  output_row = list(output_empty)
  output_row[classes.index(document[1])] = 1
  training.append([bag,output_row])


random.shuffle(training)
training = np.array(training,dtype=object)

train_x = list(training[:,0])
train_y = list(training[:,1])

In [58]:
model = Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64,activation ="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation ="softmax"))

sgd = tf.keras.optimizers.legacy.SGD(lr=0.01, decay=1e-6,momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

model.save("chatbot_model.model")
print("Model has been completed.")

Epoch 1/200
6/6 [==============================] - 0s 2ms/step - loss: 1.5680 - accuracy: 0.2069
Epoch 2/200
6/6 [==============================] - 0s 830us/step - loss: 1.6489 - accuracy: 0.2759
Epoch 3/200
6/6 [==============================] - 0s 811us/step - loss: 1.5869 - accuracy: 0.3793
Epoch 4/200
6/6 [==============================] - 0s 841us/step - loss: 1.5050 - accuracy: 0.2759
Epoch 5/200
6/6 [==============================] - 0s 962us/step - loss: 1.4869 - accuracy: 0.4138
Epoch 6/200
6/6 [==============================] - 0s 803us/step - loss: 1.3824 - accuracy: 0.5172
Epoch 7/200
6/6 [==============================] - 0s 719us/step - loss: 1.4767 - accuracy: 0.4483
Epoch 8/200
6/6 [==============================] - 0s 774us/step - loss: 1.3936 - accuracy: 0.3793
Epoch 9/200
6/6 [==============================] - 0s 829us/step - loss: 1.3272 - accuracy: 0.5517
Epoch 10/200
6/6 [==============================] - 0s 787us/step - loss: 1.2768 - accuracy: 0.6207
Epoch 11/20

INFO:tensorflow:Assets written to: chatbot_model.model/assets


Model has been completed.
